# How People Decide what they want to do
Directed graph approach

Generally people want to do a number of different things. For this I'm going to create a schema for this in a graph language that allows me to designate how much a `pop` desires to take a certain action. This will be used later when determining AI decisions. 

**Note** this notebook actualy builds the desires into the graph, overwriting existing ontology. 

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
res = db.run_query("g.V().hasLabel('pop').has('username','userbill').valueMap()")
pops = [db.clean_node(n) for n in res]
pops[0]

{'conformity': '0.197',
 'literacy': '0.822',
 'aggression': '0.384',
 'constitution': '0.615',
 'objid': '5771833830958',
 'faction_no': '2',
 'name': 'Heznius Ying',
 'isInFaction': '7100167726406',
 'industry': '0.4995',
 'wealth': '0.66075',
 'faction_loyalty': '0.49550000000000005',
 'username': 'userbill',
 'objtype': 'pop',
 'id': '5771833830958'}

Each population wants to do everything to a degree, the amount of desire to do that thing is expressed by the edge weight. 
* Attack a population
* Focus on improving literacy
* Focus on improving industry

# Desires as Objects

## Desire with targets.
Both factions and pops can have desire. Action is guided by desire based on the `max(desire.weight)`.
`desire` is an edge, the type of that desire is a property of that edge, and the edge weight is the amount of desire. The target (`node2`) is the recipient. 

Examples:
* faction wants trade with faction
* pop wants war with another pop
* pop wants faction to go to war with faction

## Desire without targets.

Desires without targets must link to an objective. That objective can be it's own node.


In [13]:
# Drop the items, if they exist. 
db.run_query("g.V().hasLabel('objective').has('username','notebook').drop()")
objectives = yaml.safe_load(open("desires.yaml"))['objectives']
data = {"nodes":objectives,'edges':[]}
db.upload_data(data)


g.addV('objective').property('type','industry').property('weight','0.5').property('leadingAttribute','wealth').property('comment','build factories, skyscrapers, infrastructure').property('username','notebook').property('objtype','objective').property('objid','7640659063021')
g.addV('objective').property('type','expansion').property('weight','0.5').property('leadingAttribute','industry').property('comment','to increase population, eventually becoming a new pop').property('username','notebook').property('objtype','objective').property('objid','5234098410041')
g.addV('objective').property('type','war').property('weight','0.4').property('leadingAttribute','aggression').property('comment','the general amount that the society wants to fight').property('username','notebook').property('objtype','objective').property('objid','1678034398160')
g.addV('objective').property('type','science').property('weight','0.5').property('leadingAttribute','literacy').property('comment','building schools, educa

# population wants to improve industry

populations want to improve industry when:
* they are not wealty
* they are at war




In [19]:
# Marginal return on 
n = 2
ind_df = pd.DataFrame(np.sort([float(p['wealth']) for p in pops]),columns=['wealth'])
ind_df['base'] = range(len(ind_df))
ind_df['desires_industry'] = ind_df['wealth'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
ind_df['desire_base'] = ind_df['wealth'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
alt.Chart(ind_df).mark_line().encode(x='base',y='desire_base').properties(title="Desire relative to the base attribute")

alt.Chart(...)

In [25]:
alt.Chart(ind_df).mark_line().encode(x='wealth:N',y='desires_industry').properties(title="Desires wealth industry relative to industry")

alt.Chart(...)

## feeding that desire to the populations


In [26]:
objectives[0],pops[0]

({'label': 'objective',
  'type': 'industry',
  'weight': 0.5,
  'leadingAttribute': 'wealth',
  'comment': 'build factories, skyscrapers, infrastructure',
  'username': 'notebook',
  'objtype': 'objective'},
 {'conformity': '0.197',
  'literacy': '0.822',
  'aggression': '0.384',
  'constitution': '0.615',
  'objid': '5771833830958',
  'faction_no': '2',
  'name': 'Heznius Ying',
  'isInFaction': '7100167726406',
  'industry': '0.4995',
  'wealth': '0.66075',
  'faction_loyalty': '0.49550000000000005',
  'username': 'userbill',
  'objtype': 'pop',
  'id': '5771833830958'})

In [ ]:
def get_desire(o,p):
        ((x+1)**(1-n) - 1)/(1-n)


for p in pops:
    for o in objectives:
        edge = {'label':'desires',
                'node1':p['objid'],
                'node2':o['type']}